In [461]:
import pandas as pd


In [807]:
busan = pd.read_csv('./busan_review.csv')
gangwondo = pd.read_csv('./gangwondo_review.csv')
jeju = pd.read_csv('./jeju_review.csv')
seoul = pd.read_csv('./seoul_review.csv')

In [808]:
# 사용하지 않기로 한 ['room_type', 'score', 'review_title'] columns 삭제 후 
# 각 DF 합침
def drop_columns(*hotels) -> pd.DataFrame:
    hotel_df = pd.DataFrame()

    for hotel in hotels:
        droped_hotel = hotel.drop(['room_type', 'score', 'review_title'], axis=1)
        hotel_df = pd.concat([hotel_df, droped_hotel], ignore_index=True)
    
    return hotel_df
    

In [809]:
# ['date', 'region', 'hotel'] 값이 NaN인 행 삭제, ['review_positive', 'review_negative'] 둘 다 NaN인 행 삭제
def drop_nan(hotel_df: pd.DataFrame, allow_nan_count: int) -> pd.DataFrame:
    thresh = len(hotel_df.columns) - allow_nan_count
    drop_nan_hotel = hotel_df.dropna(subset=['date', 'region', 'hotel']).dropna(thresh=thresh)
    
    return drop_nan_hotel

In [811]:
# column 명 통일
busan = busan.drop(['Unnamed: 8'], axis = 1)

gangwondo.columns = busan.columns
seoul.columns = busan.columns
jeju = jeju.rename(columns = {'name': 'hotel', 'review_great': 'review_positive', 'review_poor': 'review_negative'})


In [812]:
# 안쓰는 column drop
hotel_df = drop_columns(busan, gangwondo, seoul, jeju)

In [813]:
# 개행문자 변경, 날짜 형식 변경
hotel_df = hotel_df.replace('\n', ' ', regex=True)

In [814]:
# 잘못 입력된 NaN 값 drop
allow_nan_count = 1
hotel_df = drop_nan(hotel_df, allow_nan_count)

In [839]:
# 중복 데이터 삭제
hotel_df = hotel_df.drop_duplicates()
hotel_df = hotel_df[hotel_df['date'] != 'date']

In [816]:
# 의미없는 리뷰 데이터 삭제
remove_review = ['작성한 내용이 없습니다']

hotel_df = hotel_df[~hotel_df['review_positive'].isin(remove_review) & ~hotel_df['review_negative'].isin(remove_review)]

In [872]:
# date 타입 변경 및 필터링
for_datetime = hotel_df['date'].replace('년 ', '-', regex=True).replace('월', '', regex=True)
hotel_df['date'] = pd.to_datetime(for_datetime, infer_datetime_format=True)

filter_day = '2020-03'
hotel_df = hotel_df[(hotel_df['date'] >= filter_day)]

In [894]:
# 데이터 나눠서 저장

# 긍정
positive = hotel_df.drop('review_negative', axis=1).dropna().reset_index()
negative = hotel_df.drop('review_positive', axis=1).dropna().reset_index()

In [895]:
print(hotel_df.notna().sum())
print(hotel_df['review_negative'].notna().sum())
print(hotel_df['review_positive'].notna().sum())

date               44290
region             44290
hotel              44290
review_positive    43146
review_negative    36075
dtype: int64
36075
43146


In [896]:
positive.to_csv('./positive_all.csv')
negative.to_csv('./negative_all.csv')

In [906]:
# 태깅을 하기 위해 조원 인원수에 맞춰 슬라이싱 후 저장
div_num = 5
slice_range = len(positive) // div_num
for i in range(div_num):
    positive[i*slice_range: (i+1)*slice_range].to_csv(f'./positive_{i+1}.csv')


In [905]:
for i in range(2):
    print(a[i*div_num: (i+1)*div_num])

[0, 1, 2, 3, 4]
[5, 6, 7, 8, 9]
